<H3>Connecting Sql to python</H3>
keep in the mind the following <br>
# 1 - check all pip install <br>
# 2- check server name<br>
# 3- check database name<br>
# 4 - Most Driver are the same but all check that from the terminal<br>

In [1]:
pip install mysql-connector-python


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: mysql-connector-python in c:\users\doaaa\anaconda3\lib\site-packages (9.5.0)



In [2]:
!pip install SQLAlchemy pyodbc

In [3]:
from sqlalchemy import create_engine
import pyodbc

In [4]:
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=localhost;'   # change to your server name
    'DATABASE=Customer;'               # the database you restored into
    'Trusted_Connection=yes;'
)


# See available tables

In [6]:
import pandas as pd
query_tables = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE';"
tables = pd.read_sql(query_tables, conn)
tables

C:\Users\doaaa\AppData\Local\Temp\ipykernel_24592\2669543726.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables = pd.read_sql(query_tables, conn)


,TABLE_NAME
0,Dim_Customer
1,Dim_Plan
2,Dim_PaymentMethod
3,Dim_Date
4,Fact_Customer_Activity
5,sysdiagrams


# EDA (Data Exploration & Data Preprocessing)

In [7]:
# Import libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
import warnings
warnings.filterwarnings("ignore")  

In [9]:
# Check what tables exist
query_tables = """
SELECT TABLE_NAME 
FROM INFORMATION_SCHEMA.TABLES 
WHERE TABLE_TYPE='BASE TABLE' AND TABLE_NAME <> 'sysdiagrams';
"""
tables = pd.read_sql(query_tables, conn)
tables

,TABLE_NAME
0,Dim_Customer
1,Dim_Plan
2,Dim_PaymentMethod
3,Dim_Date
4,Fact_Customer_Activity


In [13]:
query = "SELECT * FROM Dim_Customer"  
df = pd.read_sql(query, conn)
print("Data Loaded. Shape:", df.shape)
df.head()

Data Loaded. Shape: (7032, 8)


,Customer_ID,Customer_Name,Gender,Senior_Citizen,Partner,Dependents,Contract_Type,Churn_Flag
0,C000001,Danielle Johnson,female,False,True,False,month-to-month,False
1,C000002,Benjamin Horton,male,False,False,False,one year,False
2,C000003,Ricky Weaver,male,False,False,False,month-to-month,True
3,C000004,Joseph Freeman,male,False,False,False,one year,False
4,C000005,Pamela Ross,female,False,False,False,month-to-month,True


In [14]:
print("Dataset shape:", df.shape)

Dataset shape: (7032, 8)


In [15]:
print("\nColumn data types:\n", df.dtypes)


Column data types:
 Customer_ID       object
Customer_Name     object
Gender            object
Senior_Citizen      bool
Partner             bool
Dependents          bool
Contract_Type     object
Churn_Flag          bool
dtype: object


In [16]:
print("\nMissing values per column:\n", df.isnull().sum())


Missing values per column:
 Customer_ID       0
Customer_Name     0
Gender            0
Senior_Citizen    0
Partner           0
Dependents        0
Contract_Type     0
Churn_Flag        0
dtype: int64


In [17]:
query = "SELECT * FROM Dim_Plan"  
df = pd.read_sql(query, conn)
print("Data Loaded. Shape:", df.shape)
df.head()

Data Loaded. Shape: (322, 11)


,Plan_ID,Internet_Service_Type,Phone_Service,Multiple_Lines,Tech_Support,Online_Security,Online_Backup,Device_Protection,Streaming_TV,Streaming_Movies,Plan_Price_Tier
0,P001,dsl,False,no phone service,no,no,yes,no,no,no,Standard
1,P002,dsl,True,no,no,yes,no,yes,no,no,Standard
2,P003,dsl,True,no,no,yes,yes,no,no,no,Standard
3,P004,dsl,False,no phone service,yes,yes,no,yes,no,no,Standard
4,P005,fiber optic,True,no,no,no,no,no,no,no,Premium


In [18]:
query = "SELECT * FROM  Dim_PaymentMethod"  
df = pd.read_sql(query, conn)
print("Data Loaded. Shape:", df.shape)
df.head()

Data Loaded. Shape: (8, 4)


,PaymentMethod_ID,Payment_Method,Paperless_Billing,Auto_Payment
0,PM001,electronic check,True,False
1,PM002,electronic check,False,False
2,PM003,mailed check,False,False
3,PM004,mailed check,True,False
4,PM005,bank transfer (automatic),False,True


In [19]:
query = "SELECT * FROM Dim_Date"  
df = pd.read_sql(query, conn)
print("Data Loaded. Shape:", df.shape)
df.head()

Data Loaded. Shape: (2191, 7)


,Date_ID,Full_Date,Year,Quarter,Month,Month_Name,Weekday
0,20180101,2018-01-01,2018,2018Q1,1,January,Monday
1,20180102,2018-01-02,2018,2018Q1,1,January,Tuesday
2,20180103,2018-01-03,2018,2018Q1,1,January,Wednesday
3,20180104,2018-01-04,2018,2018Q1,1,January,Thursday
4,20180105,2018-01-05,2018,2018Q1,1,January,Friday


In [20]:
query = "SELECT * FROM Fact_Customer_Activity"  
df = pd.read_sql(query, conn)
print("Data Loaded. Shape:", df.shape)
df.head()

Data Loaded. Shape: (7032, 11)


,Activity_ID,Customer_ID,Plan_ID,PaymentMethod_ID,Date_ID,Monthly_Charges,Total_Charges,Tenure_Months,Internet_Usage_GB,Calls_Minutes,Customer_Satisfaction_Score
0,A0000001,C000001,P001,PM002,20190117,29.850000,29.850000,1,11.440000,14,7.7
1,A0000002,C000002,P002,PM004,20211217,56.950001,1889.500000,34,16.389999,161,8.5
2,A0000003,C000003,P003,PM004,20211003,53.849998,108.150002,2,19.389999,394,4.8
3,A0000004,C000004,P004,PM006,20230218,42.299999,1840.750000,45,20.309999,19,6.8
4,A0000005,C000005,P005,PM002,20230702,70.699997,151.649994,2,69.529999,108,3.4


In [21]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Activity_ID,7032,7032,A0000001,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Customer_ID,7032,7032,C000001,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Plan_ID,7032,322,P011,1180,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PaymentMethod_ID,7032,4,PM002,2365,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Date_ID,7032.0,NaN,NaN,NaN,20205748.61661,17152.575282,20180101.0,20190625.75,20210105.0,20220702.25,20231231.0
Monthly_Charges,7032.0,NaN,NaN,NaN,64.798208,30.085974,18.25,35.587499,70.349998,89.862499,118.75
Total_Charges,7032.0,NaN,NaN,NaN,2283.300441,2266.771363,18.799999,401.449997,1397.475037,3794.737488,8684.799805
Tenure_Months,7032.0,NaN,NaN,NaN,32.421786,24.54526,1.0,9.0,29.0,55.0,72.0
Internet_Usage_GB,7032.0,NaN,NaN,NaN,46.715968,41.424615,0.0,8.965,24.38,89.057503,131.429993
Calls_Minutes,7032.0,NaN,NaN,NaN,226.912543,110.976539,0.0,166.0,239.0,303.0,558.0


In [22]:
conn.close()
print("Connection closed.")


Connection closed.
